# Classes: Intro

In [ ]:
class Base(object):
    def __init__(self, name):
        print('Base init')
        self.name = name
    def __str__(self):
        return '{}: {}'.format(self.__class__.__name__, self.name)
    def sound(self):
        print('No sound')

In [ ]:
class A(Base):
    def __init__(self, name):
        print('Init A class')
        super().__init__(name)
    def sound(self):
        print('AAAA')

In [ ]:
class B(Base):
    def __init__(self, name):
        print('Init B class')
        super().__init__(name)
    def sound(self):
        print('Ho-ho')

In [ ]:
class C(A, B):
    def sound(self):
        super(C, self).sound()

class D(B, A): 
    sound = B.sound  # B must be defined before D!

In [ ]:
c = C('C')

In [ ]:
C.__mro__

In [ ]:
c.sound()

In [ ]:
d = D('D')
D.__mro__

In [ ]:
d.sound()

In [ ]:
class MainLogicClass(object):
    def __init__(self, name):
        self.name = name
    def is_ok(self):
        return bool(self.name)
    def run(self):
        if self.is_ok():
            print('All is OK')
        else:
            print('All is bad')

In [ ]:
class MyMixin(object):
    def is_ok(self):
        return super().is_ok() and self.name.startswith('a')

In [ ]:
class MixedClass(MyMixin, MainLogicClass):
    pass

In [ ]:
v1 = MainLogicClass('test')
v1.run()

In [ ]:
v2 = MixedClass('test')
v2.run()

In [ ]:
class WrongMixedClass(MainLogicClass, MyMixin):
    pass

In [ ]:
v3 = WrongMixedClass('test')
v3.run()

## getattr vs getattribute

In [ ]:
class A(object):
    class_type = 'Snow'
    
    def __init__(self, name, **extra):
        self.name = name
        self.extra = extra
    
    def __getattr__(self, attr):
        print('getattr', attr)
        if attr in self.extra:
            return self.extra[attr]
        raise AttributeError(attr)
        
    def __setattr__(self, attr, value):
        print('Set attr', attr, value)
        if attr in ('extra', 'name'):
            self.__dict__[attr] = value
        else:
            self.extra[attr] = value
    
    def __delattr__(self, attr):
        print('Delete attribute')
        if attr not in self.extra:
            raise AttributeError
        del(self.extra[attr])
    
    def __getattribute__(self, attr):
        print('getattribute', attr)
        return super().__getattribute__(attr)

In [ ]:
a.__dict__

In [ ]:
a = A('Test')

In [ ]:
a.name

In [ ]:
a.value

In [ ]:
getattr(a, 'value')

In [ ]:
a.value = 10

In [ ]:
a.name = 12

In [ ]:
a.value

In [ ]:
a.extra

In [ ]:
del a.value

## Class creation via type

In [1]:
class A(object):
    def f_a(self):
        return 10
class B(object):
    def f_b(self):
        return 11
class C(object):
    def f_c(self):
        return 12


In [2]:
cls_map = {'a': A, 'b': B, 'c': C}

In [3]:
def method1(self):
    print(self.__class__)
def method2(self):
    print(self.__class__ * 2)

In [4]:
AB = type('AB', (A, B), {'class_name': method1})

In [5]:
i = AB()

In [6]:
i.f_c()

AttributeError: 'AB' object has no attribute 'f_c'

In [10]:
i.__class__.__bases__

(__main__.A, __main__.B)

In [11]:
i.f_a()

10

In [12]:
i.class_name()

<class '__main__.AB'>


In [14]:
def factory(code, extra_attrs=None, class_name=None):
    class_name = class_name or code.upper()
    bases = tuple(cls_map[label] for label in code)
    return type(class_name, bases, extra_attrs or {})

In [15]:
AB = factory('abc')

In [16]:
ab = AB()
ab

## underscore

In [17]:
class A(object):
    def _private(self):
        print('Private A')
    def __unique(self):
        print('Unique A')
class B(object):
    def _private(self):
        print('Private B')
    def __unique(self):
        print('Unique B')

In [18]:
class C(A, B):
    pass

In [19]:
c = C()

In [20]:
c._private()

Private A


In [21]:
c.__unique()

AttributeError: 'C' object has no attribute '__unique'

In [23]:
c._B__unique(), c._A__unique()

Unique B
Unique A


(None, None)

## class methods, static, properties

In [25]:
class E(object):
    def instance_method(self):
        print(self, 'instance')
    @classmethod
    def class_method(cls):
        print(cls, 'class')
    @staticmethod
    def static_method(self):
        print(self, 'static')
    @property
    def name(self):
        return getattr(self, '_name', '')
    @name.setter
    def name(self, value):
        self._name = value
    @name.deleter
    def name(self):
        self._name = ''

In [26]:
a = E()

In [27]:
a.instance_method()

<__main__.E object at 0x7fecac1e05c0> instance


In [28]:
a.class_method()

<class '__main__.E'> class


In [29]:
a.static_method()

TypeError: static_method() missing 1 required positional argument: 'self'

In [30]:
a.static_method('Ooops')

Ooops static


In [31]:
a.name

''

In [32]:
a.name = 'test'

In [33]:
a.name

'test'

In [34]:
del a.name
a.name

''

In [37]:
class Source(object):
    @classmethod
    def from_file(cls):
        return cls()
    @classmethod
    def from_sql(cls):
        return cls()


In [36]:
s = Source.from_sql()

## Abstraction

In [38]:
import abc

In [39]:
class AbstractSource(object, metaclass=abc.ABCMeta):
    def __init__(self, **config):
        self.config = config
    
    @abc.abstractmethod
    def load_data(self):
        """Load data
        """

In [40]:
a = AbstractSource()

TypeError: Can't instantiate abstract class AbstractSource with abstract methods load_data

In [41]:
class SQLSource(AbstractSource):
    pass

In [42]:
s = SQLSource()

TypeError: Can't instantiate abstract class SQLSource with abstract methods load_data

In [43]:
class SQLSource(AbstractSource):
    def load_data(self):
        print('Load data')

In [44]:
s = SQLSource()

In [45]:
s.load_data()

Load data
